### Pydantinc Output Parser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate
)

base_url = "http://localhost:11434/"
model = "llama3.2:latest"

llm = ChatOllama(base_url=base_url, model=model)

In [3]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

In [4]:
class Joke(BaseModel):
    """Joke to tell user"""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")
    rating: Optional[int] = Field(
        description="The rating of the joke is from 1 to 10", default=None
    )

In [5]:
parser = PydanticOutputParser(pydantic_object=Joke)

In [6]:
instruction = parser.get_format_instructions()

In [7]:
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell user", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "The rating of the joke is from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [9]:
prompt = PromptTemplate(
    template="""
    Answer the user query with a joke. Here is your formatting instruction.
    {format_instruction}

    Query: {query}
    Answer:""",
    input_variables=["query"],
    partial_variables={"format_instruction": parser.get_format_instructions()},
)

chain = prompt | llm

In [10]:
output = chain.invoke({"query": "Tell me a joke about the cat"})

In [11]:
print(output.content)

{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 8}


In [12]:
chain = prompt | llm | parser
output = chain.invoke({"query": "Tell me a joke about the dogs"})
print(output)

setup='Why did the dog go to the vet?' punchline='Because it was feeling ruff!' rating=8


### Parsing with .with_structured_output() method

In [13]:
llm.invoke("tell me a joke about the elephant")

AIMessage(content='Why did the elephant quit the circus?\n\nBecause it was tired of working for peanuts! (get it?)', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-10-24T13:00:09.223156Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4673972458, 'load_duration': 99606416, 'prompt_eval_count': 32, 'prompt_eval_duration': 3282504500, 'eval_count': 22, 'eval_duration': 1244838956, 'model_name': 'llama3.2:latest'}, id='run--7e60ab0c-2d37-40d1-87d5-d460ef89de03-0', usage_metadata={'input_tokens': 32, 'output_tokens': 22, 'total_tokens': 54})

In [16]:
output = llm.invoke("tell me a joke about the elephant")
print(output.content)

Why did the elephant quit the circus?

Because it was tired of working for peanuts.


In [17]:
structured_llm = llm.with_structured_output(Joke)
output = structured_llm.invoke("Tell me a joke about the cat")
print(output)

setup='Why did the cat join a band?' punchline='Because it wanted to be the purr-cussionist!' rating=None


### JSON Output Parser

In [18]:
from langchain_core.output_parsers import JsonOutputParser

In [19]:
parser = JsonOutputParser(pydantic_object=Joke)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell user", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "The rating of the joke is from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [20]:
prompt = PromptTemplate(
    template="""
    Answer the user query with a joke. Here is your formatting instruction.
    {format_instruction}

    Query: {query}
    Answer:""",
    input_variables=["query"],
    partial_variables={"format_instruction": parser.get_format_instructions()},
)

chain = prompt | llm
output = chain.invoke({"query": "Tell me a joke about the cat"})
print(output.content)

{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 8}


In [21]:
chain = prompt | llm | parser
output = chain.invoke({"query": "Tell me a joke about the cat"})
print(output)

{'setup': 'Why did the cat join a band?', 'punchline': 'Because it wanted to be the purr-cussionist.', 'rating': 8}


### CSV Output Parser


In [22]:
# value1, values2, values3, so on

from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [23]:
format_instruction = parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
    Answer the user query with a list of values. Here is your formatting instruction.
    {format_instruction}

    Query: {query}
    Answer:""",
    input_variables=["query"],
    partial_variables={"format_instruction": format_instruction},
)

In [25]:
chain = prompt | llm | parser

output = chain.invoke(
    {"query": "generate my website seo keywords. I have content about the Salesforce Winter 26 release."}
)
print(output)

['salesforce', 'salesforce winter 26', 'sforce', 'customer experience', 'cloud computing', 'salesforce platform', 'software as a service', 'enterprise resource planning', 'crm', 'marketing automation']
